# GT4Py code golf

This is it, you should be an expert now, are you up for a challenge then?

Look at the following Fortran code (see "An unknown computation") and port it to GT4Py, but you must follow a couple of rules to win:

- All of your code must be inside a single cell on this Jupyter notebook.
- Your solution must be implemented as a GT4Py `field_operator` named `computation`.
- The code in the single cell must run correctly as a standalone and validate against the reference (`test_computation` implemented below is the only test that counts).
- Be fair :)

To win you must hand in the working code with **the least amount of characters** (we will use `wc -m team_id.txt` to count the characters). In case of a draw, whoever **handed in first** will win (the latest recieving time for each team is the one that counts).

Note: for the hand-in you can simply send a _.txt_ file to mikael.stellio@meteoswiss.ch named `team_[NAME_OR_NUMBER_OF_YOUR_TEAM].txt`.

Good luck :)

_Hint: look in helpers.py, ../slides/, ../../QuickstartGuide.md, and the GT4Py Cheat Sheet for useful tricks._

## Divergence in ICON (example)

_WARNING: this code is just provided as an example of how a divergence operator (see exercise 3) is implemented in the ICON Fortran code. Therefore this is not part of the challenge!_

In ICON we can find a divergence in diffusion, which can be simplified down to this:

```fortran
DO jk = 1, n_levels
    DO jc = 1, n_cells

        div(jc,jk) = (vn(ieidx(jc,1),jk)*edge_length(ieidx(jc,1))*edge_orientation(jc,1) + &
                      vn(ieidx(jc,2),jk)*edge_length(ieidx(jc,2))*edge_orientation(jc,2) + &
                      vn(ieidx(jc,3),jk)*edge_length(ieidx(jc,3))*edge_orientation(jc,3)) / area(jc)
    ENDDO
ENDDO
```

where `vn` is the edge-normal velocity component and `ieidx` is the index function which returns the 3 edge neighbors of a cell for a given cell `jc`.

## An unknown computation

The following is an unknown Fortran function which we need to port to GT4Py (see rules and win conditions above):

```fortran
DO jk = 2, n_levels
    DO je = 1, n_edges

        res(je,jk) = u(icidx(je,1),jk)*EXP(2.0_sp * LOG(weights(je,1)) + &
                     u(icidx(je,2),jk)*EXP(2.0_sp * LOG(weights(je,2)) + &
                     u(icidx(je,1),jk-1)*EXP(2.0_sp * LOG(weights(je,1)) + &
                     u(icidx(je,2),jk-1)*EXP(2.0_sp * LOG(weights(je,2))
        
        res(je,jk) = res(je,jk) + &
                     MERGE( (v(icidx(je,2),jk) - v(icidx(je,1),jk)) * 0.5_sp*dt*dx_inv, 0.0_sp, v(icidx(je,1),jk) > 0.2_sp .OR. v(icidx(je,2),jk) < 0.2_sp )

    END DO
END DO
```

where `icidx` is the index function which returns the 2 neighboring cells of an edge for a given edge `je` and `_sp` stands for double precision. Also, the `MERGE` statement is the Fortran equivalent of the Numpy `where` (see [here](https://gcc.gnu.org/onlinedocs/gfortran/MERGE.html) for reference).

_Note: indexing in Fortran starts from 1 by default ;)_

In [1]:
from helpers import *

In [2]:
#@gtx.field_operator
#def computation( ...) -> gtx.Field[Dims[E, K], float]:
#    return res

In [3]:
import numpy as np

def test_computation():
    backend = None
    
    edge_domain = gtx.domain({E: n_edges, K: n_levels})
    cell_domain = gtx.domain({C: n_cells, K: n_levels})

    result_domain = gtx.domain({E: n_edges, K: (1, n_levels)})

    #u = random_field(cell_domain, allocator=backend)
    #v = random_field(cell_domain, allocator=backend)
    #weights = random_field(gtx.domain({E: n_edges, E2CDim: 2}), low=1.0, high=3.0, allocator=backend)
    
    u_l = [[-0.41925476,  0.4025132 , -0.08132916, -0.12427843,  0.06162222,  0.81609248,  0.44332044,  0.63797445, -0.80867044, -0.69018669],
           [ 0.96637576,  0.21894498, -0.04533709, -0.44048767, -0.63011496,  0.1886686 ,  0.62798334,  0.08429353,  0.47096618,  0.13884778],
           [-0.66914945, -0.04834736,  0.024724  ,  0.91453887, -0.52752048,  0.98402613, -0.82922909,  0.68939817,  0.36174235,  0.4977935 ],
           [-0.96931158, -0.35704074,  0.69910223,  0.61191723, -0.36857458,  0.00759272, -0.21629831, -0.30641916, -0.43627966, -0.81214729],
           [-0.60298068, -0.39163641, -0.10867419, -0.72647459, -0.19799336,  0.66852111,  0.24275648, -0.17529727, -0.43979549, -0.69965684],
           [-0.63443606,  0.35296596,  0.64907294, -0.82622589, -0.48893997,  0.58959133, -0.93392674, -0.62512711, -0.18327602,  0.80730414],
           [-0.27801994,  0.37376655,  0.96918038,  0.81095929,  0.60833821,  0.09491079,  0.44872607, -0.65459174, -0.50287848, -0.0270706 ],
           [ 0.56128735, -0.50409463,  0.63482779, -0.11812376, -0.25346683, -0.79829336, -0.58648483, -0.31454114,  0.2213684 ,  0.55387677],
           [-0.19507438, -0.05186781, -0.85502218,  0.29617276, -0.44873724,  0.91891433,  0.59408724, -0.47238818, -0.02649729, -0.58864205],
           [-0.25456884, -0.94869731,  0.90726796, -0.15180387, -0.58132519,  0.26605484, -0.67613423,  0.97634995,  0.21818847, -0.97146196],
           [-0.50959671, -0.51173191,  0.37606843,  0.47148728,  0.34548185, -0.81356812,  0.80701747, -0.59011   ,  0.96472873,  0.70187425],
           [-0.47326213,  0.49007083,  0.89121169,  0.49477513,  0.64772661, -0.50639655, -0.18998588,  0.76009406, -0.82793023,  0.91814754],
           [-0.58316176,  0.23783801,  0.95758203, -0.57557164,  0.50405123,  0.25620808,  0.38508494, -0.39453315, -0.65666016, -0.66882745],
           [ 0.72018869, -0.28109724,  0.80026105, -0.84412752, -0.82575294,  0.60584044, -0.36330965,  0.20177993, -0.79106445,  0.45386905],
           [-0.01287706, -0.27336313, -0.03823035,  0.68870636,  0.47039802, -0.82071317,  0.795752  , -0.7308545 ,  0.5186176 , -0.84236817],
           [-0.3868915 , -0.51427109, -0.6553808 ,  0.31067519,  0.40212695,  0.05447324, -0.77208641,  0.70935732,  0.54983598,  0.64518159],
           [-0.45362812, -0.7882774 , -0.5177619 ,  0.53253587, -0.96728743,  0.5636487 , -0.24946744, -0.61640258, -0.47021476,  0.74104259],
           [ 0.95893449,  0.21059866, -0.05806317, -0.43790629,  0.9077501 ,  0.57409485,  0.27501869, -0.69443021, -0.88379517,  0.89549625]]
    
    v_l = [[ 0.21559353, -0.82140719,  0.80987514,  0.78835473, -0.52877167, -0.21720755, -0.13528142, -0.21865895, -0.53797839, -0.00435033],
           [-0.06387437, -0.52376154, -0.808588  ,  0.79461321,  0.12180539, -0.60496154, -0.80215785,  0.7131317 , -0.57283593, -0.25557021],
           [-0.70610105, -0.18644948,  0.01774113,  0.85919864,  0.52314239, -0.69736632,  0.50682589,  0.15278057,  0.21116359, -0.72195663],
           [ 0.39661631,  0.61559241, -0.6799828 ,  0.79466661,  0.76944135, -0.30658244, -0.13448886,  0.291374  ,  0.20126647,  0.65021483],
           [-0.54746276, -0.70605826, -0.3130987 ,  0.20235097, -0.89724533,  0.90670349,  0.46954588,  0.68234244,  0.95319133,  0.40153651],
           [ 0.0417356 ,  0.57862989,  0.27952196, -0.54680893,  0.58102965,  0.10814703, -0.01682079,  0.12038435, -0.90715707, -0.86105882],
           [ 0.97558034,  0.92786463, -0.45459383, -0.8422183 ,  0.30388655, -0.42650536, -0.88767796, -0.29628196, -0.2853198 ,  0.20704893],
           [ 0.45044965, -0.58472968,  0.2633237 ,  0.42089785, -0.28531567, -0.39698999, -0.43922126,  0.04839583,  0.18408436,  0.10458418],
           [-0.19274456,  0.4161116 , -0.03661987, -0.48520473,  0.14615437, -0.14371038, -0.87373483,  0.03012754, -0.26693537, -0.73075433],
           [-0.84266794, -0.60935597,  0.14152337, -0.17997773,  0.34277244,  0.96341141,  0.95170087,  0.27770846,  0.4541819 , -0.71022273],
           [ 0.62400321, -0.9580023 ,  0.25749175,  0.70164776, -0.35267675, -0.82922283, -0.64865611, -0.14248447, -0.1471763 ,  0.08148838],
           [ 0.06737656, -0.75373261,  0.02187628, -0.43082511,  0.64556597, -0.18174601, -0.83534944,  0.51381808, -0.46617069, -0.89595382],
           [ 0.73183341,  0.46451343, -0.34538377,  0.24977504,  0.98715149,  0.26285943, -0.69073214,  0.63096342,  0.63331979, -0.0138178 ],
           [ 0.8857629 , -0.69855343,  0.03273654, -0.85548649, -0.73133719, -0.11729404,  0.04664766, -0.33268138,  0.65792975, -0.19416984],
           [-0.02857572, -0.96159578,  0.75160682, -0.1202721 ,  0.49094325,  0.13262349, -0.05272812, -0.26082363, -0.83826412,  0.3168231 ],
           [ 0.13432462, -0.08949829,  0.30859352,  0.96553772,  0.3752849 ,  0.74315262,  0.25616234,  0.68100563, -0.46833256, -0.31388459],
           [ 0.51602823, -0.47333421, -0.37935271,  0.03232378,  0.03534582, -0.71327359,  0.17036575,  0.700366  , -0.7262425 , -0.90437802],
           [-0.49941518, -0.28846961, -0.55367433,  0.92018006,  0.40542253,  0.02414554,  0.01360208,  0.09854335, -0.31231703,  0.16373168]]
    
    weights_l = [[1.62903447, 2.43464156],
                 [2.13691961, 1.04325911],
                 [2.72535425, 2.26391145],
                 [1.56389818, 2.91125246],
                 [2.84422554, 1.18755853],
                 [1.25547346, 2.2999846 ],
                 [2.46633474, 1.75007945],
                 [2.19780815, 2.85054682],
                 [1.21740796, 1.23887092],
                 [2.5877351 , 2.7372262 ],
                 [2.68030821, 2.84749312],
                 [1.94596901, 1.71191764],
                 [2.38288854, 1.75176413],
                 [2.81730484, 2.53854831],
                 [1.20782132, 1.20890432],
                 [1.40569919, 1.01661455],
                 [2.48938783, 2.02020822],
                 [1.50757319, 1.45918605],
                 [1.54467092, 2.2045187 ],
                 [1.40996396, 2.81527127],
                 [2.47316797, 1.01142844],
                 [1.13308243, 2.59253985],
                 [2.24802007, 2.1250144 ],
                 [1.78405413, 2.72036151],
                 [1.49449367, 1.33094726],
                 [2.98483604, 2.02534186],
                 [2.22404992, 2.75863704]]
    
    u = gtx.as_field(cell_domain, np.array(u_l), dtype=float, allocator=backend)
    v = gtx.as_field(cell_domain, np.array(v_l), dtype=float, allocator=backend)
    
    weights = gtx.as_field(gtx.domain({E: n_edges, E2CDim: 2}), np.array(weights_l), dtype=float, allocator=backend)
    
    dt = 0.2
    dx_inv = 1.2

    e2c_connectivity = gtx.NeighborTableOffsetProvider(
        e2c_table, E, C, 2, has_skip_values=False
    )

    res = gtx.zeros(result_domain, allocator=backend)

    #computation(
    #    u,
    #    v,
    #    weights,
    #    dt,
    #    dx_inv,
    #    out=res,
    #    offset_provider={E2C.value: e2c_connectivity, Koff.value: K},
    #)
    
    ref_l = [[-5.29822174e+00, -6.14089819e+00, -2.56760525e+00,  4.05884595e+00,  5.03572653e+00, -9.11466168e-01,  2.49766447e+00,  7.01921493e+00,  3.06870103e+00],
             [-1.52003713e+00,  1.66017620e+00,  4.88765649e-01, -2.12632856e-02,  3.60440481e+00,  5.52395728e+00,  4.36873222e+00, -1.58781618e+00, -8.20319416e+00],
             [-1.36251836e+01,  2.50333103e+00,  1.45594134e+01,  3.76146870e+00, -3.88380801e-01, -7.56791942e-01, -4.61581910e+00, -1.27843891e-01, -5.03204654e+00],
             [-7.62055258e+00, -1.05930791e+01, -1.15447884e+00, -6.31352036e+00, -4.51367303e+00,  4.77686072e+00, -5.59804803e+00, -7.86987378e+00,  3.70898403e+00],
             [ 8.16421369e+00,  7.58294715e-01, -5.17901497e+00, -1.00868261e+01, -2.90754977e+00,  7.89157344e+00,  5.85349115e+00,  3.62437710e+00,  3.32619836e+00],
             [-5.27400040e+00, -2.27510985e+00, -4.81227986e+00, -5.03334643e+00,  3.87252431e+00,  6.80569770e+00,  2.06120614e+00, -3.52285049e+00, -8.39014111e+00],
             [-7.94616501e-01,  2.54916157e-01,  4.20162813e+00,  3.77905912e+00,  7.40197944e+00,  3.48306002e+00, -2.14163741e+00,  1.49731314e+00,  5.37051112e+00],
             [-5.75288662e+00,  8.02807518e+00,  2.92878048e+00, -8.81014943e+00,  3.11960295e+00, -2.11304977e+00, -1.33475961e+01, -1.62558506e+00,  9.20578946e+00],
             [ 1.32474207e+00,  1.79523162e+00, -1.15289740e+00, -3.60523912e+00, -4.14206917e-01,  7.76099323e-01, -1.40831449e+00, -4.57916090e-01,  1.78889482e+00],
             [-8.12690448e+00,  1.23795234e+01,  2.19201805e+01,  1.22076015e+01,  2.83736795e+00,  2.58519551e+00, -5.11325930e+00, -1.37040169e+01, -1.23837067e+01],
             [-9.25296102e+00,  9.38341657e+00,  1.89935518e+01,  4.25660387e+00,  2.49589693e+00,  5.80502700e-01,  9.55264488e-01,  1.37025049e+00, -1.00249594e+01],
             [-5.28022684e+00, -2.83604972e+00,  1.18636192e+00, -3.24691888e+00,  5.12111744e-02,  2.66215714e+00,  1.46380459e+00,  2.97487384e+00, -4.65772186e+00],
             [-5.45752736e+00, -2.43966463e+00, -3.13028290e+00, -6.31613509e+00, -7.12225302e-01,  8.15888125e-01, -2.45798777e+00, -3.87080368e+00, -4.12664693e+00],
             [-6.17251623e+00,  1.62709893e-01,  9.59671842e+00,  2.30726041e+00, -1.14163602e+01, -1.10586166e+01, -5.77673378e+00,  1.63484579e+00,  1.68904784e+01],
             [-1.57924612e+00,  1.76086906e+00,  3.83558530e+00,  3.18568874e+00,  2.93511771e-01,  8.12184199e-01,  3.51315498e-02, -1.12449136e+00,  1.64748221e+00],
             [-8.30900383e-01,  1.00480659e+00, -9.20233742e-01, -2.80861610e+00,  6.54593648e-01,  7.80461824e-01, -2.96572938e+00, -2.03617079e+00,  6.12960321e-01],
             [-1.60209444e+00,  2.43225051e-02,  2.19985332e+00,  3.71737519e+00,  3.48594952e+00,  6.53865060e+00,  3.08092438e+00, -3.39238309e+00, -3.46367562e+00],
             [ 1.45495458e-01,  3.20921343e+00,  4.02322256e+00,  1.58809955e+00, -2.06366364e+00, -4.67758142e+00, -8.33943040e-01, -4.34229839e-01,  1.83398764e+00],
             [-4.54924281e+00,  5.65234543e+00,  3.65907690e+00, -2.01950872e+00,  2.85848239e+00,  1.94107690e+00,  7.12790400e-01, -2.23702212e+00, -8.15548898e+00],
             [-7.89537601e+00, -6.89387809e+00, -1.88672368e+00,  5.43388675e+00,  5.18792946e+00, -4.41273476e+00, -5.09953209e-01,  7.75808537e+00,  6.80033810e+00],
             [-5.90948677e+00, -7.41984039e+00, -1.57328080e+00,  5.55952520e+00,  2.36119085e+00, -4.45192570e+00, -4.30316433e-01,  7.44042367e+00,  6.97820369e+00],
             [ 1.67112384e+00,  3.28829077e+00,  6.06462090e-01, -1.02202578e+01, -1.99362285e+00,  1.70513948e+00, -8.30025043e-01, -3.47977220e+00, -1.59746666e-01],
             [-3.36203925e+00, -3.32347111e+00, -4.84314898e-02, -1.03100979e+01, -3.00557189e+00,  2.65924237e+00, -4.72630295e+00, -8.05092844e+00, -5.66300226e-01],
             [-1.04022114e+01, -5.86440440e+00,  1.29912010e+00, -3.55917629e+00, -6.84419868e-01,  4.46952720e+00, -6.42949962e+00, -1.15503040e+01, -2.32146911e+00],
             [-4.94452910e-01,  2.53314705e+00,  4.28515024e+00,  4.58649724e+00, -2.67168860e-01, -1.50568022e+00,  1.29534684e+00, -5.72124887e-01, -3.71997048e-01],
             [ 2.32802715e+00, -2.30699467e+00,  3.76077937e+00,  1.22437876e+01,  2.94448098e+00,  3.26864634e+00, -1.09911830e+00, -8.30165296e+00, -2.85474662e+00],
             [ 1.11213701e+01,  3.65842860e+00, -3.99134752e+00, -4.68434531e+00,  1.02061541e+01,  7.65750898e+00, -3.93899783e+00, -1.50416842e+01, -1.53590837e+00]]
    
    ref_np = np.array(ref_l)
    
    assert np.allclose(res.asnumpy(), ref_np)

In [ ]:
test_computation()
print("Test successful")